# Pairwise Boosting: Filter and Thresholds Design

## Libraries and Configurations

Import configuration files

In [ ]:
from configparser import ConfigParser

config = ConfigParser()
config.read("../config.ini")

Import **data libraries**

In [ ]:
import pandas as pd

Import **other libraries**

In [ ]:
from rich.progress import Progress
from rich import traceback

traceback.install()

## Playground

In [3]:
# data_point = '0000000000001100010000010110111001110100011011110101111101001000011011110111010001010011011100000110111101110100000000010000010000000010000001000000101100010110001100100000100000001100000100100001100000100100001100000100100001100000011011000000001100000001000000010010110100011010111011110000000100010011111111111111111100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000010111111000011001001001011110001100100000011001111111010111111110110001000000011111110101111111101100010000000111101110100000111000000000101000011110010000010000000000000101000000000000111111100001011000000000000000000001010100000100000000001000000000000000000000000000000000000010010000011111111000000110000001000000000010001011101110100001010010100000110111110011010000101100000001100000001000000010110010100000001000000011101110100001000100011001111110111110000000000010000000100000010000000010000000010111101111100100110101110100001'

data_point = "000000000000110001000001011"

In [4]:
len(data_point)

27

In [1]:
def generate_bitmasks(length):
    """
    Generate a set of bitmasks for binary strings of a given length.

    Parameters:
        length (int): The length of the binary strings.

    Returns:
        set: A set containing the bitmasks.
    """
    bitmasks = set()
    for i in range(2**length):
        bitmask = format(i, "b").zfill(length)
        bitmasks.add(bitmask)
    return bitmasks


# Example usage:
bitmasks = generate_bitmasks(10)
print(bitmasks)

{'0001111000', '1110010010', '1000000011', '1100000001', '1111011100', '1101111110', '1110001010', '0000001010', '1001101001', '0010001111', '0000101011', '1111101000', '0011011001', '0010001001', '0010100011', '0100110010', '0111101011', '1111110110', '0001011001', '1101111000', '0110101010', '1011110001', '1100101110', '1100001011', '0101110101', '0100010000', '1100011111', '1111101101', '0101110000', '0010010001', '0100010011', '0110010111', '0111010111', '0010110001', '1111110000', '1001111110', '0010101110', '1011111111', '0011001000', '0100010111', '1011001001', '1011001100', '1100111010', '1101111011', '1110011000', '1000011110', '0111010110', '1010110011', '1100000101', '1100100110', '0011000010', '0111010100', '1101010010', '1100111100', '1010111101', '0010101011', '1001100100', '1011100010', '0001010010', '0001000001', '0000100111', '0100000011', '0111011011', '0010001100', '0100001000', '0000011001', '0110110101', '0000110011', '1001111001', '1101110011', '1110001000', '1001

In [3]:
def generate_thresholds(bitmasks):
    """
    Generate thresholds for each bitmask in a set.

    Parameters:
        bitmasks (set): A set containing the bitmasks.

    Returns:
        dict: A dictionary where keys are bitmasks and values are sets of thresholds.
    """
    thresholds_dict = {}
    for bitmask in bitmasks:
        max_ones = bitmask.count("1")
        thresholds = set(range(max_ones + 1))
        thresholds_dict[bitmask] = thresholds
    return thresholds_dict


# Example usage:
bitmasks = generate_bitmasks(10)
thresholds_dict = generate_thresholds(bitmasks)
for bitmask, thresholds in thresholds_dict.items():
    print(f"Bitmask: {bitmask}, Thresholds: {thresholds}")

Bitmask: 0001111000, Thresholds: {0, 1, 2, 3, 4}
Bitmask: 1110010010, Thresholds: {0, 1, 2, 3, 4, 5}
Bitmask: 1000000011, Thresholds: {0, 1, 2, 3}
Bitmask: 1100000001, Thresholds: {0, 1, 2, 3}
Bitmask: 1111011100, Thresholds: {0, 1, 2, 3, 4, 5, 6, 7}
Bitmask: 1101111110, Thresholds: {0, 1, 2, 3, 4, 5, 6, 7, 8}
Bitmask: 1110001010, Thresholds: {0, 1, 2, 3, 4, 5}
Bitmask: 0000001010, Thresholds: {0, 1, 2}
Bitmask: 1001101001, Thresholds: {0, 1, 2, 3, 4, 5}
Bitmask: 0010001111, Thresholds: {0, 1, 2, 3, 4, 5}
Bitmask: 0000101011, Thresholds: {0, 1, 2, 3, 4}
Bitmask: 1111101000, Thresholds: {0, 1, 2, 3, 4, 5, 6}
Bitmask: 0011011001, Thresholds: {0, 1, 2, 3, 4, 5}
Bitmask: 0010001001, Thresholds: {0, 1, 2, 3}
Bitmask: 0010100011, Thresholds: {0, 1, 2, 3, 4}
Bitmask: 0100110010, Thresholds: {0, 1, 2, 3, 4}
Bitmask: 0111101011, Thresholds: {0, 1, 2, 3, 4, 5, 6, 7}
Bitmask: 1111110110, Thresholds: {0, 1, 2, 3, 4, 5, 6, 7, 8}
Bitmask: 0001011001, Thresholds: {0, 1, 2, 3, 4}
Bitmask: 1101111000, 

# BAMBOO Implementation

In [1]:
from scipy.spatial import distance

In [11]:
probe1 = "00000011"
probe2 = "00001111"

In [12]:
def hamming_distance(str1, str2):
    """
    Compute the Hamming distance between two strings.

    Parameters:
        str1 (str): The first string.
        str2 (str): The second string.

    Returns:
        int: The Hamming distance between the two strings.
    """
    if len(str1) != len(str2):
        raise ValueError("The two strings must have equal length.")

    distance = sum(c1 != c2 for c1, c2 in zip(str1, str2))
    return distance


hamming_distance = hamming_distance(list(probe1), list(probe2))
print(hamming_distance)

2


In [15]:
def generate_bitmaps(total_length):
    """
    Generate bitmaps of 8 consecutive 1s within a longer string of 0s.

    Parameters:
        total_length (int): The total length of the combined bitmaps.

    Returns:
        list: A list containing the bitmaps.
    """
    if total_length % 8 != 0:
        raise ValueError("Total length must be a multiple of 8.")

    num_bitmaps = total_length // 8
    bitmaps = [
        "0" * (i * 8) + "1" * 8 + "0" * ((num_bitmaps - i - 1) * 8)
        for i in range(num_bitmaps)
    ]
    return bitmaps


# Example usage:
total_length = 1024
bitmaps = generate_bitmaps(total_length)
for bitmap in bitmaps:
    print(bitmap)

...


In [49]:
import pandas as pd

# Create a DataFrame from the bitmaps list
df = pd.DataFrame({"bitmaps": bitmaps})

# Save the DataFrame to a CSV file
df.to_csv("../../data/filters/bitmaps.csv", index=False)

In [25]:
packet_IE = "0000000000001100010000010110111001110100011011110101111101001000011011110111010001010011011100000110111101110100000000010000010000000010000001000000101100010110001100100000100000001100000100100001100000100100001100000100100001100000011011000000001100000001000000010010110100011010111011110000000100010011111111111111111100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000010111111000011001001001011110001100100000011001111111010111111110110001000000011111110101111111101100010000000111101110100000111000000000101000011110010000010000000000000101000000000000111111100001011000000000000000000001010100000100000000001000000000000000000000000000000000000010010000011111111000000110000001000000000010001011101110100001010010100000110111110011010000101100000001100000001000000010110010100000001000000011101110100001000100011001111110111110000000000010000000100000010000000010000000010111101111100100110101110100001"

packet_IE2 = "0000000000001100010000010110111001110100011011110101111101001000011011110111010001010011011100000110111101110100000000010000010000000010000001000000101100010110001100100000100000001100000100100001100000100100001100000100100001100000011011000000001100000001000000010010110100011010111011110000000100010011111111111111111100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000010111111000011001001001011110001100100000011001111111010111111110110001000000011111110101111111101100010000000111101110100000111000000000101000011110010000010000000000000101000000000000111111100001011000000000000000000001010100000100000000001000000000000000000000000000000000000010010000011111111000000110000001000000000010001011101110100001010010100000110111110011010000101100000001100000001000000010110010100000001000000011101110100001000100011001111110111110000000000010000000100000010000000010000000010111101111100100110101110100001"

In [22]:
bitwise_and_result = int(packet_IE, 2) & int(bitmaps[6], 2)
bitwise_and_result_binary = bin(bitwise_and_result)[2:].zfill(len(packet_IE))
bitwise_and_result_binary

'000000000000000000000000000000000000000000000000010111110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [26]:
bitwise_and_result2 = int(packet_IE2, 2) & int(bitmaps[6], 2)
bitwise_and_result_binary2 = bin(bitwise_and_result2)[2:].zfill(len(packet_IE2))
bitwise_and_result_binary2

'000000000000000000000000000000000000000000000000010111110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [42]:
def hamming_distance(str1, str2):
    """
    Calculate the Hamming distance between two binary strings.

    Parameters:
        str1 (str): The first binary string.
        str2 (str): The second binary string.

    Returns:
        int: The Hamming distance between the two strings.
    """
    distance = 0
    for char1, char2 in zip(str1, str2):
        if char1 != char2:
            distance += 1
    return distance


hamming_distance(bitwise_and_result_binary, bitwise_and_result_binary2)

0